In [5]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np

from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

In [6]:
df = pd.read_parquet('/Users/harshitgupta/Desktop/vs /VS-Data-Den/dates_grouped_data.parquet')

In [3]:
df.columns

Index(['PTNR_RM_CODE', 'WEEK', 'DATE_FIELD', 'SALES', 'FLAG'], dtype='object')

In [4]:
df.head()

,PTNR_RM_CODE,WEEK,DATE_FIELD,SALES,FLAG
0,0.0,1,2021-01-31,1497279.48,Active
1,0.0,1,2021-02-01,1352352.29,Active
2,0.0,1,2021-02-02,1336891.59,Active
3,0.0,1,2021-02-03,1333770.96,Active
4,0.0,1,2021-02-04,1468008.14,Active


In [7]:
df = df.set_index('DATE_FIELD')

In [8]:
df.head()

,PTNR_RM_CODE,WEEK,SALES,FLAG
DATE_FIELD,,,,
2021-01-31,0.0,1,1497279.48,Active
2021-02-01,0.0,1,1352352.29,Active
2021-02-02,0.0,1,1336891.59,Active
2021-02-03,0.0,1,1333770.96,Active
2021-02-04,0.0,1,1468008.14,Active


In [9]:
df['FLAG'] = df['FLAG'].map({'Inactive': 0, 'Active': 1})

In [11]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error


In [18]:
def train_test_VAR(df, partner_code):
    # Filter data for the specific PTNR_RM_CODE
    data = df[df['PTNR_RM_CODE'] == partner_code]

    # Drop unnecessary columns
    data = data.drop(['PTNR_RM_CODE'], axis=1)

    # Remove constant columns
    constant_columns = data.columns[data.nunique() == 1]
    data = data.drop(constant_columns, axis=1)

    # Check if there are enough observations to fit the model
    if len(data) <= 12:
        print(f"Not enough data for PTNR_RM_CODE: {partner_code}")
        return None, None

    # Create train-test split (80% train, 20% test)
    train_size = int(len(data) * 0.8)
    train, test = data.iloc[:train_size], data.iloc[train_size:]

    # Initialize VAR model
    model = VAR(train)

    # Fit VAR model with max lag of 12
    model_fitted = model.fit(maxlags=12)

    # Forecast
    forecast = model_fitted.forecast(model_fitted.endog, steps=len(test))

    # Convert forecast results to DataFrame
    forecast_df = pd.DataFrame(forecast, index=test.index, columns=test.columns)

    # Evaluate forecast performance
    rmse = np.sqrt(mean_squared_error(test['SALES'], forecast_df['SALES']))

    return forecast_df, rmse


In [19]:
results = []

In [20]:
for partner_code in df['PTNR_RM_CODE'].unique():
    forecast, rmse = train_test_VAR(df, partner_code)
    results.append({'PTNR_RM_CODE': partner_code, 'Forecast': forecast, 'RMSE': rmse})

/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provid

ValueError: Only gave one variable to VAR